<a href="https://colab.research.google.com/github/CHRISTIANDAGATTI/AEyCD---Grupo-9/blob/main/Entregable_parte_1_22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Diplomatura en Ciencia de Datos, Aprendizaje Automático y sus Aplicaciones**

**Exploración y Curación de Datos**

*Edición 2022*

----

# Trabajo práctico entregable - parte 1

En esta notebook, vamos a cargar el conjunto de datos de [la compentencia Kaggle](https://www.kaggle.com/dansbecker/melbourne-housing-snapshot) sobre estimación de precios de ventas de propiedades en Melbourne, Australia.

Utilizaremos el conjunto de datos reducido producido por [DanB](https://www.kaggle.com/dansbecker). Hemos subido una copia a un servidor de la Universidad Nacional de Córdoba para facilitar su acceso remoto.

In [114]:
!pip install sqlalchemy

In [115]:
!pip install decouple

In [116]:
!pip install missingno

In [117]:
!pip install plotly

In [118]:
!pip install config

In [119]:
import matplotlib.pyplot as plt
import numpy
import pandas

import seaborn
seaborn.set_context('talk')


import config
from sqlalchemy import create_engine, text

In [120]:
import plotly
plotly.__version__
# Make sure it's 4.14.3

'5.5.0'

In [121]:
melb_df = pandas.read_csv(
    'https://cs.famaf.unc.edu.ar/~mteruel/datasets/diplodatos/melb_data.csv')
melb_df[:3]

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra,-37.7996,144.9984,Northern Metropolitan,4019.0
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0
2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019.0


## Ejercicio 1 SQL: 

1. Crear una base de datos en SQLite utilizando la libreria [SQLalchemy](https://stackoverflow.com/questions/2268050/execute-sql-from-file-in-sqlalchemy).
https://docs.sqlalchemy.org/en/14/core/engines.html#sqlite

2. Ingestar los datos provistos en 'https://cs.famaf.unc.edu.ar/~mteruel/datasets/diplodatos/melb_data.csv' en una tabla y el dataset generado en clase con datos de airbnb y sus precios por codigo postal en otra.

3. Implementar consultas en SQL que respondan con la siguiente información:

    - cantidad de registros totales por ciudad.
    - cantidad de registros totales por barrio y ciudad.

4. Combinar los datasets de ambas tablas ingestadas utilizando el comando JOIN de SQL  para obtener un resultado similar a lo realizado con Pandas en clase.  



**1- Creación de base de datos:**

In [122]:
engine = create_engine('sqlite:///sysarmy.sqlite3', echo=True)

**2- Creación de tablas:**

In [123]:
melb_df.to_sql('Melbo', con=engine, if_exists="replace")

2022-05-15 03:21:30,896 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Melbo")
2022-05-15 03:21:30,897 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-05-15 03:21:30,900 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Melbo")
2022-05-15 03:21:30,902 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-05-15 03:21:30,907 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2022-05-15 03:21:30,908 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-05-15 03:21:30,911 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Melbo")
2022-05-15 03:21:30,913 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-05-15 03:21:30,923 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2022-05-15 03:21:30,928 INFO sqlalchemy.engine.Engine [raw sql] ('Melbo',)
2022-05-15 03:21:30,931 INFO sqlalchemy.engine.Engine PRAGMA main.foreign_key_list("Melbo")
2022-05-1

In [124]:
interesting_cols = [
  'description', 'neighborhood_overview',
  'street', 'neighborhood', 'city', 'suburb', 'state', 'zipcode',
  'price', 'weekly_price', 'monthly_price',
  'latitude', 'longitude',
]
airbnb_df = pandas.read_csv(
    'https://cs.famaf.unc.edu.ar/~mteruel/datasets/diplodatos/cleansed_listings_dec18.csv',
    usecols=interesting_cols)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (35) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [125]:
airbnb_df.to_sql('Airbnb', con=engine, if_exists="replace")

2022-05-15 03:21:38,466 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Airbnb")
2022-05-15 03:21:38,468 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-05-15 03:21:38,472 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Airbnb")
2022-05-15 03:21:38,475 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-05-15 03:21:38,478 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2022-05-15 03:21:38,479 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-05-15 03:21:38,482 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Airbnb")
2022-05-15 03:21:38,484 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-05-15 03:21:38,488 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2022-05-15 03:21:38,490 INFO sqlalchemy.engine.Engine [raw sql] ('Airbnb',)
2022-05-15 03:21:38,493 INFO sqlalchemy.engine.Engine PRAGMA main.foreign_key_list("Airbnb")
2022

**A traves de "to_numeric" llevamos todos los registros al mismo tipo de datos**

In [126]:
airbnb_df['zipcode'] = pandas.to_numeric(airbnb_df.zipcode, errors='coerce')

In [127]:
airbnb_df.zipcode.value_counts().head(10)

3000.0    3367
3006.0    1268
3182.0    1135
3141.0     881
3004.0     728
3121.0     628
3008.0     616
3053.0     538
3181.0     460
3056.0     446
Name: zipcode, dtype: int64

**3. Consultas SQL:**

**Cantidad de registros totales por ciudad:**


(Guardamos strings en variables y luego pandas lo convierte en una sentencia sql)

In [128]:
query1 = "SELECT city, COUNT(*) FROM Airbnb GROUP BY city ORDER BY COUNT(city) desc"
query2 = "SELECT city, neighborhood, COUNT(city) FROM Airbnb GROUP BY city, neighborhood  ORDER BY city"

In [129]:
with engine.connect() as con:
      rs = con.execute(query1)
      print(query1)
      for row in rs:
          print(row)

2022-05-15 03:21:40,051 INFO sqlalchemy.engine.Engine SELECT city, COUNT(*) FROM Airbnb GROUP BY city ORDER BY COUNT(city) desc
2022-05-15 03:21:40,054 INFO sqlalchemy.engine.Engine [raw sql] ()
SELECT city, COUNT(*) FROM Airbnb GROUP BY city ORDER BY COUNT(city) desc
('Melbourne', 7368)
('Port Phillip', 2808)
('Yarra', 2049)
('Stonnington', 1621)
('Moreland', 967)
('Yarra Ranges', 771)
('Darebin', 698)
('Boroondara', 664)
('Glen Eira', 631)
('Whitehorse', 614)
('Monash', 571)
('Maribyrnong', 436)
('Wyndham', 426)
('Bayside', 375)
('Moonee Valley', 344)
('Manningham', 313)
('Kingston', 309)
('Hobsons Bay', 239)
('Banyule', 203)
('Frankston', 177)
('Knox', 175)
('Hume', 170)
('Casey', 153)
('Greater Dandenong', 147)
('Whittlesea', 137)
('Cardinia', 123)
('Maroondah', 115)
('Brimbank', 108)
('Melton', 95)
('Nillumbik', 88)


In [130]:

with engine.connect() as con:
      rs = con.execute(query2)
      print(query2)
      for row in rs:
          print(row)

2022-05-15 03:21:40,110 INFO sqlalchemy.engine.Engine SELECT city, neighborhood, COUNT(city) FROM Airbnb GROUP BY city, neighborhood  ORDER BY city
2022-05-15 03:21:40,115 INFO sqlalchemy.engine.Engine [raw sql] ()
SELECT city, neighborhood, COUNT(city) FROM Airbnb GROUP BY city, neighborhood  ORDER BY city
('Banyule', None, 174)
('Banyule', 'Alphington', 16)
('Banyule', 'Preston', 8)
('Banyule', 'Thornbury', 5)
('Bayside', None, 170)
('Bayside', 'Bentleigh', 1)
('Bayside', 'Brighton', 203)
('Bayside', 'Elwood', 1)
('Boroondara', None, 30)
('Boroondara', 'Ashburton', 14)
('Boroondara', 'Balwyn', 70)
('Boroondara', 'Balwyn North', 40)
('Boroondara', 'Camberwell', 82)
('Boroondara', 'Canterbury', 24)
('Boroondara', 'Glen Iris', 32)
('Boroondara', 'Hawthorn', 261)
('Boroondara', 'Kew', 109)
('Boroondara', 'Malvern', 2)
('Brimbank', None, 108)
('Cardinia', None, 123)
('Casey', None, 153)
('Darebin', None, 112)
('Darebin', 'Alphington', 22)
('Darebin', 'Coburg', 4)
('Darebin', 'Northcote', 

**Cantidad de registros totales por barrio y ciudad:**

**4.JOIN de Datasets por SQL:**

Controlamos de que tengamos suficientes registros en común como para agregar información relevante a traves del JOIN por Código Postal.

In [131]:
intersection = numpy.intersect1d(
    airbnb_df.zipcode.values, melb_df.Postcode.values, assume_unique=False)
print("Airbnb unique zipcodes", len(airbnb_df.zipcode.unique()))
print("Sales unique zipcodes", len(melb_df.Postcode.unique()))
print("Common zipcodes", len(intersection))

Airbnb unique zipcodes 248
Sales unique zipcodes 198
Common zipcodes 191


Calculamos el porentaje de el total de registros de Postcode o Zipcode que interaccionan (cruzan) entre los distintos Dataset respecto el total de registros.

In [132]:
print('Records in Sales df with corresponding zipcode form Airbnb df',
      melb_df.Postcode.isin(intersection).sum() / len(melb_df))
print('Records in Airbnb df with corresponding zipcode form Sales df',
      airbnb_df.zipcode.isin(intersection).sum() / len(airbnb_df))

Records in Sales df with corresponding zipcode form Airbnb df 0.9985272459499264
Records in Airbnb df with corresponding zipcode form Sales df 0.9302904564315353


Definimos columnas de interes del Dataset de Airbnb y calculamos ciertas métricas de los registros agrupados por Zipcode.

In [133]:
relevant_cols = ['price', 'weekly_price', 'monthly_price', 'zipcode']
## Option 1
# Pass a dictionary where the keys are the original columns to aggregate and
# the values are the operations (or list of operations).
airbnb_price_by_zipcode = airbnb_df[relevant_cols].groupby('zipcode')\
  .agg({'price': ['mean', 'count'], 'weekly_price': 'mean',
        'monthly_price': 'mean'})\
  .reset_index()
# Flatten the two level columns
airbnb_price_by_zipcode.columns = [
  ' '.join(col).strip()
  for col in airbnb_price_by_zipcode.columns.values]
# Rename columns
airbnb_price_by_zipcode = airbnb_price_by_zipcode.rename(
    columns={'price mean': 'airbnb_price_mean',
             'price count': 'airbnb_record_count',
             'weekly_price mean': 'airbnb_weekly_price_mean',
             'monthly_price mean': 'airbnb_monthly_price_mean'}
)

In [134]:
airbnb_price_by_zipcode.head()

,zipcode,airbnb_price_mean,airbnb_record_count,airbnb_weekly_price_mean,airbnb_monthly_price_mean
0,2010.0,40.000000,1,NaN,NaN
1,2134.0,50.000000,1,NaN,NaN
2,2582.0,104.000000,1,NaN,NaN
3,3000.0,150.504307,3367,918.738956,3407.204651
4,3001.0,132.500000,2,NaN,NaN


In [135]:
airbnb_price_by_zipcode.to_sql('Metrica', con=engine, if_exists="replace")

2022-05-15 03:21:40,326 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Metrica")
2022-05-15 03:21:40,331 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-05-15 03:21:40,334 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Metrica")
2022-05-15 03:21:40,337 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-05-15 03:21:40,340 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2022-05-15 03:21:40,343 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-05-15 03:21:40,347 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Metrica")
2022-05-15 03:21:40,350 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-05-15 03:21:40,353 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2022-05-15 03:21:40,355 INFO sqlalchemy.engine.Engine [raw sql] ('Metrica',)
2022-05-15 03:21:40,358 INFO sqlalchemy.engine.Engine PRAGMA main.foreign_key_list("Metrica")

Generamos y ejecutamos la sentencia de SQL para unir mediante LEFT JOIN las tablas "Melbo" y "Métricas" (generada a partir del Dataset de Airbnb.

Primeramente ejecutamos un conteo de registros del Leftjoin (Desde tabla Melbo) para comparar y controlar la cantidad de registros de la tabla resultante:

In [153]:
query3 = "SELECT COUNT() FROM Melbo LEFT JOIN Metrica ON Metrica.zipcode=Melbo.Postcode"

In [155]:
with engine.connect() as con:
      rs = con.execute(query3)
      print(query3)
      for row in rs:
          print(row)

2022-05-15 03:29:46,254 INFO sqlalchemy.engine.Engine SELECT COUNT() FROM Melbo LEFT JOIN Metrica ON Metrica.zipcode=Melbo.Postcode
2022-05-15 03:29:46,256 INFO sqlalchemy.engine.Engine [raw sql] ()
SELECT COUNT() FROM Melbo LEFT JOIN Metrica ON Metrica.zipcode=Melbo.Postcode
(13580,)


In [152]:
len(melb_df)

13580

In [166]:
query4 = "SELECT * FROM Melbo LEFT JOIN Metrica ON Metrica.zipcode=Melbo.Postcode"

In [164]:
# with engine.connect() as con:
#     rs = con.execute(query4)
#     print(query4)
#     for row in rs:
#        print(row)

## Ejercicio 2: 

1. Seleccionar un subconjunto de columnas que les parezcan relevantes al problema de predicción del valor de la propiedad. Justificar las columnas seleccionadas y las que no lo fueron.
 - Eliminar los valores extremos que no sean relevantes para la predicción de valores de las propiedades.

2. Agregar información adicional respectiva al entorno de una propiedad a partir del [conjunto de datos de AirBnB](https://www.kaggle.com/tylerx/melbourne-airbnb-open-data?select=cleansed_listings_dec18.csv) utilizado en el práctico. 
  1. Seleccionar qué variables agregar y qué combinaciones aplicar a cada una. Por ejemplo, pueden utilizar solo la columna `price`, o aplicar múltiples transformaciones como la mediana o el mínimo.
  1. Utilizar la variable zipcode para unir los conjuntos de datos. Sólo incluir los zipcodes que tengan una cantidad mínima de registros (a elección) como para que la información agregada sea relevante.
  2. Investigar al menos otras 2 variables que puedan servir para combinar los datos, y justificar si serían adecuadas o no. Pueden asumir que cuentan con la ayuda de anotadores expertos para encontrar equivalencias entre barrios o direcciones, o que cuentan con algoritmos para encontrar las n ubicaciones más cercanas a una propiedad a partir de sus coordenadas geográficas. **NO** es necesario que realicen la implementación.

Pueden leer otras columnas del conjunto de AirBnB además de las que están en `interesting_cols`, si les parecen relevantes.

## Ejercicio 3:

Crear y guardar un nuevo conjunto de datos con todas las transformaciones realizadas anteriormente.

## Ejercicios opcionales:

1. Armar un script en python (archivo .py) [ETL](https://towardsdatascience.com/what-to-log-from-python-etl-pipelines-9e0cfe29950e) que corra los pasos de extraccion, transformacion y carga, armando una funcion para cada etapa del proceso y luego un main que corra todos los pasos requeridos.

2. Armar un DAG en Apache Airflow que corra el ETL. (https://airflow.apache.org/docs/apache-airflow/stable/tutorial.html)